In [62]:
import random
import numpy as np
import enum
import time
from termcolor import colored

In [63]:
class LiftsQueueCommand(enum.Enum):
    UP = 'up'
    DOWN = 'down'
    END1 = 'end1'
    END2 = 'end2'
    
class LiftsPassState(enum.Enum):
    NONE = 0
    SELECTED1 = 1
    SELECTED2 = 2

In [64]:
GLOBAL_QUEUE = None
commands = {}

def global_queue_reset():
    global GLOBAL_QUEUE
    GLOBAL_QUEUE = [(command[0], LiftsQueueCommand(command[1])) for command in commands.keys()] + [None]

def global_queue_add_end_commands(command_key):
    if command_key:
        global GLOBAL_QUEUE
        global commands
        command_key = (command_key[0], command_key[1].value)
        if command_key in commands.keys():
            for command in commands[command_key][::-1]:
                add_command = (command[0], LiftsQueueCommand(command[1]))
                if add_command not in GLOBAL_QUEUE:
                    GLOBAL_QUEUE.insert(0, add_command)

In [65]:
# пример для тестирования
floors = 8
commands = {
    (2, "up"): [(5, "end1")],
    (3, "up"): [(4, "end1")],
    (2, "down"): [(1, "end2")],
    (1, "up"): [(3, "end1",), (2, "end1")]
}
#логика: лифт-1: 1, 2, 3, 4, 5
#        лифт-2: 2, 1
global_queue_reset()

In [66]:
class LiftsQueueHandler:

    def __init__(self, floors: int):
        self.floors = floors
        self.states_dca = {}
        self.state = None

        self.queue_iterator = None
        self.queue_selector = None
        self.last_popped_command = None

    def reset_global_queue(self):
        global_queue_reset()
        global GLOBAL_QUEUE
        self.commands_queue = GLOBAL_QUEUE

    def run(self, start_state: tuple):
        self.state = (*start_state, LiftsPassState.NONE, 0)
        for command_idx in range(len(self.commands_queue)):
            self.queue_iterator = command_idx
            item = self.states_dca[self.state][self.commands_queue[command_idx]]
            # print(self.state, self.commands_queue[command_idx], item["state"])
            self.state = item["state"]
            item["select_action"]()
        self.last_popped_command = self.commands_queue.pop(self.queue_selector)
        return self.state
        #

    def print_commands_queue(self):
        print([(command[0], command[1].value) if command else None for command in self.commands_queue])

    def select_current_item(self):
        self.queue_selector = self.queue_iterator

    def generate_easy_dca(self):
        for floor1 in range(1, self.floors + 1):
            for floor2 in range(1, self.floors + 1):
                for pass_state in LiftsPassState:
                    for selected_floor in range(0, self.floors + 1):
                        state_dict = self.states_dca[floor1, floor2, pass_state, selected_floor] = {}
                        state_dict[None] = {
                            'state': (floor1, floor2, pass_state, selected_floor),
                            'select_action': lambda: None
                        }

                        for command_floor in range(1, self.floors + 1):

                            # Если лифт не выбран
                            if pass_state == LiftsPassState.NONE:
                                state_dict[command_floor, LiftsQueueCommand.END1] = {
                                    'state': (floor1, floor2, LiftsPassState.SELECTED1, command_floor),
                                    'select_action': self.select_current_item
                                }
                                state_dict[command_floor, LiftsQueueCommand.END2] = {
                                    'state': (floor1, floor2, LiftsPassState.SELECTED2, command_floor),
                                    'select_action': self.select_current_item
                                }
                                if abs(floor1 - command_floor) <= abs(floor2 - command_floor):
                                    for commands_action in (LiftsQueueCommand.UP, LiftsQueueCommand.DOWN):
                                        state_dict[command_floor, commands_action] = {
                                            'state': (floor1, floor2, LiftsPassState.SELECTED1, command_floor),
                                            'select_action': self.select_current_item
                                        }
                                else:
                                    for commands_action in (LiftsQueueCommand.UP, LiftsQueueCommand.DOWN):
                                        state_dict[command_floor, commands_action] = {
                                            'state': (floor1, floor2, LiftsPassState.SELECTED2, command_floor),
                                            'select_action': self.select_current_item
                                        }

                            else:
                                used_floor = floor1 if pass_state == LiftsPassState.SELECTED1 else floor2
                                # (15, 1, <LiftsPassState.SELECTED2: 2>, 1) (11, <LiftsQueueCommand.DOWN: 'down'>) (15, 1, <LiftsPassState.SELECTED2: 2>, 11)
                                # 1 11 1
                                # при движении вверх; если этаж команды лежит по пути
                                if used_floor <= command_floor < selected_floor:
                                    if command_floor != self.floors:
                                        state_dict[command_floor, LiftsQueueCommand.UP] = {
                                            'state': (floor1, floor2, pass_state, command_floor),
                                            'select_action': self.select_current_item
                                        }
                                    if command_floor != 1:
                                        state_dict[command_floor, LiftsQueueCommand.DOWN] = {
                                            'state': (floor1, floor2, pass_state, selected_floor),
                                            'select_action': lambda: None
                                        }
                                    state_dict[command_floor, LiftsQueueCommand.END1] = {
                                            'state': (floor1, floor2, pass_state, command_floor if pass_state == LiftsPassState.SELECTED1 else selected_floor),
                                            'select_action': self.select_current_item
                                    }
                                    state_dict[command_floor, LiftsQueueCommand.END2] = {
                                            'state': (floor1, floor2, pass_state, command_floor if pass_state == LiftsPassState.SELECTED2 else selected_floor),
                                            'select_action': self.select_current_item
                                    }
                                # при движении вниз; если этаж команды лежит по пути
                                elif used_floor >= command_floor > selected_floor:
                                    if command_floor != self.floors:
                                        state_dict[command_floor, LiftsQueueCommand.UP] = {
                                            'state': (floor1, floor2, pass_state, selected_floor),
                                            'select_action': lambda: None
                                        }
                                    if command_floor != 1:
                                        state_dict[command_floor, LiftsQueueCommand.DOWN] = {
                                            'state': (floor1, floor2, pass_state, command_floor),
                                            'select_action': self.select_current_item
                                        }
                                    state_dict[command_floor, LiftsQueueCommand.END1] = {
                                            'state': (floor1, floor2, pass_state, command_floor if pass_state == LiftsPassState.SELECTED1 else selected_floor),
                                            'select_action': self.select_current_item
                                    }
                                    state_dict[command_floor, LiftsQueueCommand.END2] = {
                                            'state': (floor1, floor2, pass_state, command_floor if pass_state == LiftsPassState.SELECTED2 else selected_floor),
                                            'select_action': self.select_current_item
                                    }

                                else:
                                    if command_floor != self.floors:
                                        state_dict[command_floor, LiftsQueueCommand.UP] = {
                                            'state': (floor1, floor2, pass_state, selected_floor),
                                            'select_action': lambda: None
                                        }
                                    if command_floor != 1:
                                        state_dict[command_floor, LiftsQueueCommand.DOWN] = {
                                            'state': (floor1, floor2, pass_state, selected_floor),
                                            'select_action': lambda: None
                                        }
                                    state_dict[command_floor, LiftsQueueCommand.END1] = {
                                            'state': (floor1, floor2, pass_state, selected_floor),
                                            'select_action': lambda: None
                                    }   
                                    state_dict[command_floor, LiftsQueueCommand.END2] = {
                                            'state': (floor1, floor2, pass_state, selected_floor),
                                            'select_action': lambda: None
                                    }                              

                                

In [67]:
class LiftActions(enum.Enum):
    START = 0
    UP1 = 1
    UP2 = 2
    DOWN1 = 3
    DOWN2 = 4
    OPEN1 = 5
    OPEN2 = 6
    CLOSE1 = 7
    CLOSE2 = 8

    UPDATE_COMMANDS = 9
    NEXT_COMMAND = 10


In [68]:
class TwoLifts:
    def __init__(self, floors: int, start_state: tuple, global_queue: list = [], print_movments: bool = True):
        self.floors = floors
        self.states_dca = {}
        self.state = (*start_state, LiftsPassState.NONE)
        self.current_command = None
        self.lifts_queue_handler = LiftsQueueHandler(self.floors)
        # self.lifts_queue_handler.set_global_queue()
        
        self.print_movments = print_movments

        self.actions = {
            LiftActions.UP1: lambda: (self.plus_move_count(), print(f"Лифты: [{self.state[0]}, {self.state[1]}]->[{self.state[0] + 1}, {self.state[1]}]") if self.print_movments else None),
            LiftActions.DOWN1: lambda: (self.plus_move_count(), print(f"Лифты: [{self.state[0]}, {self.state[1]}]->[{self.state[0] - 1}, {self.state[1]}]") if self.print_movments else None),
            LiftActions.OPEN1: lambda: (print("Лифт 1: открыт") if self.print_movments else None),
            LiftActions.CLOSE1: lambda: (print("Лифт 1: закрыт") if self.print_movments else None),
            
            LiftActions.UP2: lambda: (self.plus_move_count(), print(f"Лифты: [{self.state[0]}, {self.state[1]}]->[{self.state[0]}, {self.state[1] + 1}]") if self.print_movments else None),
            LiftActions.DOWN2: lambda: (self.plus_move_count(), print(f"Лифты: [{self.state[0]}, {self.state[1]}]->[{self.state[0]}, {self.state[1] - 1}]") if self.print_movments else None),
            LiftActions.OPEN2: lambda: (print("Лифт 2: открыт") if self.print_movments else None),
            LiftActions.CLOSE2: lambda: (print("Лифт 2: закрыт") if self.print_movments else None),

            LiftActions.UPDATE_COMMANDS: lambda: global_queue_add_end_commands(self.lifts_queue_handler.last_popped_command),
            LiftActions.NEXT_COMMAND: lambda: self.set_current_command(self.lifts_queue_handler.run((self.state[0], self.state[1])))
        }

    def set_state(self, state: tuple):
        self.state = (*state, LiftsPassState.NONE)
    def plus_move_count(self):
        self.move_count += 1

    def set_current_command(self, new_command):
        self.current_command = new_command

    def run(self):
        self.move_count = 0
        self.current_command = self.lifts_queue_handler.run((self.state[0], self.state[1]))
        while self.current_command[3]:
            next_item = self.states_dca[self.state][self.current_command[2], self.current_command[3]]

            next_item["action"]()
            self.state = next_item["state"]
        
        print(colored(f"Всего операций перемещения: {self.move_count}", "light_green"))
            
    def exit(self):
        return 


    def generate_dca(self):
        self.lifts_queue_handler.generate_easy_dca()

        for floor1 in range(1, self.floors + 1):
            for floor2 in range(1, self.floors + 1):
                for pass_state in LiftsPassState:
                    self.states_dca[floor1, floor2, pass_state] = {}

                    self.states_dca[floor1, floor2, pass_state][LiftsPassState.NONE, 0] = {
                        "state": self.state,
                        "action": self.exit()
                    }
                    for next_floor in range(1, self.floors + 1):
                        #NONE
                        self.states_dca[floor1, floor2, pass_state][LiftsPassState.NONE, next_floor] = {
                            "state": None,
                            "action": lambda: None
                        }
                        #SELECTED1
                        if floor1 < next_floor:
                            self.states_dca[floor1, floor2, pass_state][LiftsPassState.SELECTED1, next_floor] = {
                                "state": (floor1 + 1, floor2, LiftsPassState.SELECTED1),
                                "action": self.actions[LiftActions.UP1]
                            }
                        elif floor1 > next_floor:
                            self.states_dca[floor1, floor2, pass_state][LiftsPassState.SELECTED1, next_floor] = {
                                "state": (floor1 - 1, floor2, LiftsPassState.SELECTED1),
                                "action": self.actions[LiftActions.DOWN1]
                            }
                        else:
                            self.states_dca[floor1, floor2, pass_state][LiftsPassState.SELECTED1, next_floor] = {
                                "state": (floor1, floor2, LiftsPassState.NONE),
                                "action": lambda: (self.actions[LiftActions.OPEN1](), self.actions[LiftActions.NEXT_COMMAND](), self.actions[LiftActions.UPDATE_COMMANDS](), self.actions[LiftActions.CLOSE1]())
                            }
                        #SELECTED2
                        if floor2 < next_floor:
                            self.states_dca[floor1, floor2, pass_state][LiftsPassState.SELECTED2, next_floor] = {
                                "state": (floor1, floor2 + 1, LiftsPassState.SELECTED2),
                                "action": self.actions[LiftActions.UP2]
                            }
                        elif floor2 > next_floor:
                            self.states_dca[floor1, floor2, pass_state][LiftsPassState.SELECTED2, next_floor] = {
                                "state": (floor1, floor2 - 1, LiftsPassState.SELECTED2),
                                "action": self.actions[LiftActions.DOWN2]
                            }
                        else:
                            self.states_dca[floor1, floor2, pass_state][LiftsPassState.SELECTED2, next_floor] = {
                                "state": (floor1, floor2, LiftsPassState.NONE),
                                "action": lambda: (self.actions[LiftActions.OPEN2](), self.actions[LiftActions.NEXT_COMMAND](), self.actions[LiftActions.UPDATE_COMMANDS](), self.actions[LiftActions.CLOSE2]())
                            }



## Проверка на условиях lab3-DKA-v1

### Подготовка очереди команд с проверкой

In [69]:
floors = 15
start_state = (9, 6)
#actions = [[2, 10], [6, 8], [2, 7], [1, 3], [11, 7], [3, 7], [3, 11], [9, 12], [11, 2]]

commands = {
    (2, "up"): [(10, "end2"), (7, "end2")],
    (6, "up"): [(8, "end2")],
    (1, "up"): [(3, "end1")],
    (11, "down"): [(7, "end2"), (2, "end2")],
    (3, "up"): [(7, "end2"), (11, "end2")],
    (9, "up"): [(12, "end2")]
}


In [70]:
liftsQH = LiftsQueueHandler(floors)
liftsQH.generate_easy_dca()

In [71]:
liftsQH.reset_global_queue()

print("1----")
liftsQH.print_commands_queue()
print(liftsQH.run((9, 6))[2:], liftsQH.last_popped_command)
global_queue_add_end_commands((2, LiftsQueueCommand("up"))) 

print("2----")
liftsQH.print_commands_queue()
print(liftsQH.run((9, 2))[2:], liftsQH.last_popped_command)
global_queue_add_end_commands((3, LiftsQueueCommand("up")))

print("3----")
liftsQH.print_commands_queue()
print(liftsQH.run((9, 3))[2:], liftsQH.last_popped_command)
global_queue_add_end_commands((6, LiftsQueueCommand("up")))

print("4----")
liftsQH.print_commands_queue()
print(liftsQH.run((9, 6))[2:], liftsQH.last_popped_command)

print("5----")
liftsQH.print_commands_queue()
print(liftsQH.run((9, 7))[2:], liftsQH.last_popped_command)

print("6----")
liftsQH.print_commands_queue()
print(liftsQH.run((9, 8))[2:], liftsQH.last_popped_command)
global_queue_add_end_commands((9, LiftsQueueCommand("up")))

print("7----")
liftsQH.print_commands_queue()
print(liftsQH.run((9, 9))[2:], liftsQH.last_popped_command)

print("8----")
liftsQH.print_commands_queue()
print(liftsQH.run((9, 10))[2:], liftsQH.last_popped_command)

print("9----")
liftsQH.print_commands_queue()
print(liftsQH.run((9, 11))[2:], liftsQH.last_popped_command)

print("10----")
liftsQH.print_commands_queue()
print(liftsQH.run((9, 12))[2:], liftsQH.last_popped_command)
global_queue_add_end_commands((1, LiftsQueueCommand("up")))

print("11----")
liftsQH.print_commands_queue()
print(liftsQH.run((1, 12))[2:], liftsQH.last_popped_command)

print("12----")
liftsQH.print_commands_queue()
print(liftsQH.run((3, 12))[2:], liftsQH.last_popped_command)
global_queue_add_end_commands((11, LiftsQueueCommand("down")))

print("13----")
liftsQH.print_commands_queue()
print(liftsQH.run((3, 11))[2:], liftsQH.last_popped_command)

print("14----")
liftsQH.print_commands_queue()
print(liftsQH.run((3, 7))[2:], liftsQH.last_popped_command)

print("15----")
liftsQH.print_commands_queue()
print(liftsQH.run((3, 2))[2:], liftsQH.last_popped_command)

1----
[(2, 'up'), (6, 'up'), (1, 'up'), (11, 'down'), (3, 'up'), (9, 'up'), None]
(<LiftsPassState.SELECTED2: 2>, 2) (2, <LiftsQueueCommand.UP: 'up'>)
2----
[(10, 'end2'), (7, 'end2'), (6, 'up'), (1, 'up'), (11, 'down'), (3, 'up'), (9, 'up'), None]
(<LiftsPassState.SELECTED2: 2>, 3) (3, <LiftsQueueCommand.UP: 'up'>)
3----
[(11, 'end2'), (10, 'end2'), (7, 'end2'), (6, 'up'), (1, 'up'), (11, 'down'), (9, 'up'), None]
(<LiftsPassState.SELECTED2: 2>, 6) (6, <LiftsQueueCommand.UP: 'up'>)
4----
[(8, 'end2'), (11, 'end2'), (10, 'end2'), (7, 'end2'), (1, 'up'), (11, 'down'), (9, 'up'), None]
(<LiftsPassState.SELECTED2: 2>, 7) (7, <LiftsQueueCommand.END2: 'end2'>)
5----
[(8, 'end2'), (11, 'end2'), (10, 'end2'), (1, 'up'), (11, 'down'), (9, 'up'), None]
(<LiftsPassState.SELECTED2: 2>, 8) (8, <LiftsQueueCommand.END2: 'end2'>)
6----
[(11, 'end2'), (10, 'end2'), (1, 'up'), (11, 'down'), (9, 'up'), None]
(<LiftsPassState.SELECTED2: 2>, 9) (9, <LiftsQueueCommand.UP: 'up'>)
7----
[(12, 'end2'), (11, '

In [72]:
lifts = TwoLifts(floors=floors, start_state=start_state, print_movments=True)
lifts.generate_dca()

In [75]:
lifts.lifts_queue_handler.reset_global_queue()
lifts.lifts_queue_handler.print_commands_queue()
lifts.set_state(start_state)
lifts.run()

[(2, 'up'), (6, 'up'), (1, 'up'), (11, 'down'), (3, 'up'), (9, 'up'), None]
Лифты: [9, 6]->[9, 5]
Лифты: [9, 5]->[9, 4]
Лифты: [9, 4]->[9, 3]
Лифты: [9, 3]->[9, 2]
Лифт 2: открыт
Лифт 2: закрыт
Лифты: [9, 2]->[8, 2]
Лифты: [8, 2]->[7, 2]
Лифты: [7, 2]->[6, 2]
Лифт 1: открыт
Лифт 1: закрыт
Лифты: [6, 2]->[6, 3]
Лифт 2: открыт
Лифт 2: закрыт
Лифты: [6, 3]->[6, 4]
Лифты: [6, 4]->[6, 5]
Лифты: [6, 5]->[6, 6]
Лифты: [6, 6]->[6, 7]
Лифт 2: открыт
Лифт 2: закрыт
Лифты: [6, 7]->[6, 8]
Лифт 2: открыт
Лифт 2: закрыт
Лифты: [6, 8]->[6, 9]
Лифт 2: открыт
Лифт 2: закрыт
Лифты: [6, 9]->[6, 10]
Лифты: [6, 10]->[6, 11]
Лифт 2: открыт
Лифт 2: закрыт
Лифты: [6, 11]->[6, 12]
Лифт 2: открыт
Лифт 2: закрыт
Лифты: [6, 12]->[5, 12]
Лифты: [5, 12]->[4, 12]
Лифты: [4, 12]->[3, 12]
Лифты: [3, 12]->[2, 12]
Лифты: [2, 12]->[1, 12]
Лифт 1: открыт
Лифт 1: закрыт
Лифты: [1, 12]->[2, 12]
Лифты: [2, 12]->[3, 12]
Лифт 1: открыт
Лифт 1: закрыт
Лифты: [3, 12]->[3, 11]
Лифт 2: открыт
Лифт 2: закрыт
Лифты: [3, 11]->[3, 10]

Результат стал лучше по сравнению с решением без подбора людей (lab3-DKA-v1), где суммарно получилось 65 перемещений лифтов по этажам.